# pytorch 신경망을 위한 자동미분(auto grad) 패키지

### tensor.requires_grad 속성을 true로 설정하면 이에 대한 모든 연산 추적
### 계산이 끝난 후 .backword()를 호출하여 자동으로 모든 기울기를 구할 수 있음, tensor의 기울기는 .grad 속성에 누적됨

### tensor가 기록을 추적하는 것을 중지하려면 .detatch() 호출, 계산기록으로부터 분리, 이후 연산들을 추적하는 것을 방지할 수 있음
### 기록을 추적방지 하기 위해( 메모리 사용 방지) with torch.no.grad()로 코드 블럭을 감쌀 수 있음, 특히 모델 평가할 때 유용


In [2]:
import torch as nn

x = nn.ones(2,2, requires_grad= True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [3]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [4]:
print(y.grad_fn)

In [5]:
## 추적하는 텐서를 이용한 연산을 결과로 받는 텐서도 grad_fn를 받음

z = y * y * 3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [6]:
a = nn.randn(2,2)
a = ((a*3) / (a-1))
print(a.requires_grad)

## in_place 함수를 이용하여 속성 변경
a.requires_grad_(True)
print(a.requires_grad)

b = (a * a).sum()
print(b.grad_fn)

False
True


In [7]:
print(out)

tensor(27., grad_fn=<MeanBackward0>)


## backword() 사용  

In [8]:
out.backward()

In [10]:
"""
y = x + 2
z = y * y * 3
out = z.mean()

out =  3(x + 2)^2 / 4
    = (3 /4) * ( x + 2) ^2

dout / dx = (6 / 4) *(x + 2) 
          = 1.5 * (x + 2)

x = 1일 때 기울기 dout / dx = 4.5
"""



print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [11]:
## torch.autograd는 벡터-야코비안 곱을 계산하는 엔진
## 야코비안 행렬 = https://en.wikipedia.org/wiki/Jacobian_matrix_and_determinant

## 예제 

x = nn.randn(5, requires_grad=True)

y = x * 2

while y.data.norm() < 1000:
    y = y * 2
    
print(y)

tensor([ -860.5074,   193.7361,   531.6152,  -513.6329, -1470.3262],
       grad_fn=<MulBackward0>)


In [15]:
y.data.norm()

tensor(1867.1633)

In [13]:
v = nn.tensor([0.1, 1.0, 0.0001, 0.5, 0.8], dtype=nn.float)
y.backward(v)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01, 5.1200e+02, 8.1920e+02])


In [16]:
## with torch.no_grad()로 코드를 감사 연산 기록 추적을 막음

print(x.requires_grad)
print((x ** 2).requires_grad)

with nn.no_grad():
    print((x ** 2).requires_grad)

True
True
False


In [17]:
## .detach() 사용

print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
print(x.eq(y).all()) ## x와 y의 값이 같냐  torch.eq() 함수

True
False
tensor(True)


In [18]:
x.eq(y)

tensor([True, True, True, True, True])